# Observational study 

In [6]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotnine
from networkx import nx
from plotnine import ggplot, aes, geom_bar
from functions import *
import spacy

In [16]:
from compare import *

ModuleNotFoundError: No module named 'googlesearch'

## Matching : 

In [13]:
data = pd.DataFrame(columns = ['Elon','Name','date','Close','Volume','MarketCap','Popularity'])

#Data is our data frame with all the companies with first column 1 or 0 if Elon Musk talks about it.
# after,Name of the company and the other four covariates 
def compare (data,company_quotes, date, companies): 
    companies.append(company_quotes)
    company = compareCompanies(companies,date)
    company ['Elon']= 0
    company ['compare'] = company_quotes
    (company[company['Name']==company_quote])['Elon']=1
    (company[company['Name']==company_quote])['compare']= 'None'
    controled = controled[['Elon', 'Name', 'date', 'Close','Volume','MarketCap','Popularity']]
    data.concate(controled)
     
    
#Call this function for all our functions and a set of company that will represent the controled one.


In [14]:
#Propensity score : 
def add_propensityscore (companies, date):#so that if we want to change only the first line don't have to do it
    #With a column of one and zero if elon musk talked about you 
    #company = company_quote.extend(controled)
    data = compareCompanies(company,date)
    #Normalise the features 
    data['Close'] = (data['Close'] - data['Close'].mean())/lalonde_data['Close'].std()
    data['Volume']= (data['Volume']-data['Volume'].mean())/data['Volume'].std()
    data['MarketCap']= (data['MarketCap']-data['MarketCap'].mean())/data['MarketCap'].std()
    data['Popularity']= (data['Popularity']-data['Popularity'].mean())/data['Popularity'].std()
    #Create the model
    mod = smf.logit(formula='elon ~ Close+ Volume + MarketCap + Popularity', data=data)
    res = mod.fit()
    data['Propensity_score'] = res.predict()

In [15]:
def get_similarity(propensity_score1, propensity_score2):
    '''Calculate similarity for instances with given propensity scores'''
    return 1-np.abs(propensity_score1-propensity_score2)

In [ ]:
def matching(companies):
    treatment_df = data[data['elon'] == 1]
    control_df   = data[data['elon'] == 0]
    # Create an empty undirected graph
    G = nx.Graph()
    # Loop through all the pairs of instances
    for control_id, control_row in control_df.iterrows():
        for treatment_id, treatment_row in treatment_df.iterrows():
            if (controle_row['compare']== treatment_id) and (controle_row['date']== treatment_row['date']) : 
                # Calculate the similarity 
                similarity = get_similarity(control_row['Propensity_score'],
                                    treatment_row['Propensity_score'])
                # Add an edge between the two instances weighted by the similarity between them
                G.add_weighted_edges_from([(control_id, treatment_id, similarity)])
    # Generate and return the maximum weight matching on the generated graph
    matching = nx.max_weight_matching(G)
    return matching

In [ ]:
matching = matching(data)
matched = [i[0] for i in list(matching)]+ [i[1]for in list(matching)]
balanced = data.iloc[matched]


## After matching : 
